##### Boilerplate

In [1]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

##### Pure LCEL - Single Session

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

prompt_template = ChatPromptTemplate.from_messages([
    ('system', "You are a helpful assistant. Always answer my questions shortly & briefly."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

conversation = []


def chat_with_bot(payload):
    input = payload['input']
    history = payload['history']
    conversation.append(HumanMessage(content=input))
    res = chain.invoke({"input": input, "history": conversation})
    conversation.append(AIMessage(content=res))
    return res


print(conversation)
res = chat_with_bot({"input": "What is google?", "history": conversation})
print(res)
print(conversation)

res = chat_with_bot({"input": "Where is it located?", "history": conversation})
print(res)
print(conversation)

[]
Google is a search engine.
[HumanMessage(content='What is google?', additional_kwargs={}, response_metadata={}), AIMessage(content='Google is a search engine.', additional_kwargs={}, response_metadata={})]
Google's headquarters, called the Googleplex, is located in Mountain View, California.
[HumanMessage(content='What is google?', additional_kwargs={}, response_metadata={}), AIMessage(content='Google is a search engine.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Where is it located?', additional_kwargs={}, response_metadata={}), AIMessage(content="Google's headquarters, called the Googleplex, is located in Mountain View, California.", additional_kwargs={}, response_metadata={})]


##### Pure LCEL - Multi Session

##### RunnableWithMessageHistory